### 2D frame dataset generation for CIDRE-based retrospective flat/dark-field model estimation

Run the cells below to extract a suitable dataset of 2D images from the 3D microscopy image stacks included in *source*.
Frames exported to *outdir* will satisfy the following inclusion criteria:
* *bg_lvl* (approximate) dark background intensity
* *bg_rel_thr*  relative number of sub-threshold (*bg_lvl*) background pixels  
* *mean_rel_thr*  mean intensity relative to data type maximum

In [ ]:
import numpy as np
import random
import tifffile as tiff

from math import ceil

from pathlib import Path


##### I/O image paths

In [ ]:
# path to TIFF z-stacks
wv = 638

base = Path('/mnt/NASoneScratch/quadspim/')
sample = f'resliced_theta20_cam_l_dir_l_250414_Broca_I38_3col_638nm'
outdir = Path(f'/mnt/NASoneScratch/michele/mic_dset')

# channel axis for RGB z-stacks (ch_ax = 1 when True, ch_ax = 3 otherwise)
ch_first = True

# objective name and fluorescence emission wavelength [nm]
obj = 'qspim_dskwd'

# input formats
formats = ["_l.tiff"]


##### Export options

In [ ]:
# inclusion/exclusion criteria
bg_lvl = 500
bg_rel_thr = 0.333
mean_rel_thr = 0.005

# maximum output size [B]
max_dset_size = 100e9

# minimum z-stride between exported frames [μm]
z_strd_um = 17
px_sz_z = 3.4

# force output x-size (set to None otherwise)
x_shp_out = 10000


##### Evaluate I/O directory content

In [ ]:
source = base / sample

if not isinstance(wv, tuple) and not isinstance(wv, list):
    wv = (wv,)
    
num_slc = np.zeros((len(wv),), dtype=int)

ch_ax = 0 if ch_first else 2

# get current number of 2D frames in outdir
outdir = Path(outdir)
wavedirs = []
for c, w in enumerate(wv):
    if w != -1:
        wavedir = outdir / f'{obj}_{w}nm'

        if not (wavedir).is_dir():
            wavedir.mkdir(parents=True, exist_ok=True)
        else:
            num_slc[c] = sum(1 for file in wavedir.iterdir() if file.suffix.lower() in ['.tif', '.tiff'])
            
        wavedirs.append(wavedir)

# get all z-stacks file paths in source
stack_lst = []
source = Path(source)  # Ensure source is a Path object
for dirpath in source.glob('*'):
    if dirpath.is_file() and any(str(dirpath).lower().endswith(fmt) for fmt in formats):
        stack_lst.append(dirpath)

random.shuffle(stack_lst)

print(f"Output directory path\n{outdir}\n\nProcessed image stacks")
for s in stack_lst:
    print(s)


##### Export 2D frames imposing the required inclusion criteria

In [ ]:
from skimage.transform import resize


ds_size = np.zeros((len(wv),))
num_out = 1
num_stk = len(stack_lst)

z_strd = ceil(z_strd_um / px_sz_z)

for f in stack_lst:
    prc_progress = 100 * (num_out / num_stk)
    print('\nProcessing image stack {0}/{1}\t{2:0.1f}%'.format(num_out, num_stk, prc_progress), end='\r')

    # read image stack
    img = tiff.imread(f)
    img_max = np.iinfo(img.dtype).max

    # loop over z-slices
    for z in range(0, img.shape[0], z_strd):
        
        # read single frame
        slc = img[z, ...]

        # add channel axis when missing
        if slc.ndim == 2:
            slc = slc[np.newaxis, ...] if ch_first else slc[..., np.newaxis]

        # loop over channels
        for c in range(slc.shape[ch_ax]):           
            if wv[c] > 0:

                # check criteria and save "valid" z-slice to TIFF
                slc = slc[c, :, :] if ch_first else slc[..., c]
                if np.count_nonzero(slc > bg_lvl) / np.size(slc) > bg_rel_thr \
                    and np.mean(slc[slc != 0]) > mean_rel_thr * img_max:

                    # force output shape
                    if x_shp_out is not None:
                        slc = resize(slc, output_shape=(slc.shape[0], x_shp_out), anti_aliasing=True, preserve_range=True)

                    tiff.imwrite(wavedirs[c] / f'{num_slc[c]}.tiff', slc, compression='zlib')

                    # update total dataset size and slice counter
                    ds_size[c] += slc.itemsize * slc.size
                    num_slc[c] += 1
    
                # break if all channels reached max size
                if np.all(ds_size) >= max_dset_size:
                    break

    # increase processed stack counter
    num_out += 1

# print exported dataset information
print(f"\n\nExported data size  (objective: {obj})")
for c, w in enumerate(wv):
    if w != -1:
        print(f"- {w} nm: {num_slc[c]} images")
